In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import foregrounds as fg
import experiments as exp
from cosmo import CosmoCalc
import hlc

KeyboardInterrupt: 

In [ ]:
components = ['radio', 'cib', 'tsz', 'tsz_cib']
experiment = 's4wide'
l, cl_residual, res_ilc_dic, weights = hlc.residuals_and_weights(components = components, experiment = experiment)

In [ ]:
plt.plot(l, weights[0], label = '27 GHz', color = 'violet')
plt.plot(l, weights[1], label = '39 GHz', color = 'orange')
plt.plot(l, weights[2], label = '93 GHz', color = 'blue')
plt.plot(l, weights[3], label = '145 GHz', color = 'magenta')
plt.plot(l, weights[4], label = '225 GHz', color = 'brown')
plt.plot(l, weights[5], label = '278 GHz', color = 'deepskyblue')
tot = np.asarray(weights[0])+np.asarray(weights[1])+np.asarray(weights[2])+np.asarray(weights[3])+np.asarray(weights[4])+np.asarray(weights[5])
plt.plot(l, tot, label = 'Sum', color = 'green', ls = '--')
plt.xlim(0, 5000)
plt.ylim(-1., 2.)
plt.legend(ncol=4)

In [ ]:
l, cl = CosmoCalc().cmb_power_spectrum()
l, cl_ksz = fg.get_foreground_power_spt('kSZ')
plt.subplot(xscale = 'log', yscale = 'log')
plt.plot(l, cl, label = 'CMB', color = 'red')
plt.plot(l, cl_residual, label = 'Residual', color = 'blue')
plt.plot(l, cl+cl_residual, label = 'Total', color = 'black', ls = '--')
plt.xlim(100, 10000)
plt.ylim(1e-7,1e1)
plt.legend(ncol=3)

In [ ]:
import lensing_estimator
import stats

sims_for_covariance =[]
l, cl = CosmoCalc().cmb_power_spectrum()
maps_clus = sims.cmb_mock_data(10, [240,0.25,240,0.25], l, cl)
_, dipole_profile, _ = lensing_estimator.get_dipole_profile([240,0.25,240,0.25], maps_clus, maps_rand, l, cl, cl_noise = cl)
sims_for_covariance.append(dipole_profile) 
nber_pixels = 10/0.25
cov, corr = stats.covariance_matrix(sims_for_covariance, int(nber_pixels))
print(cov)

In [ ]:
import tools
grid, _ = tools.make_grid([240,0.25,240,0.25])
X,Y = grid
plt.imshow(1.5*X)
plt.colorbar()

In [ ]:
import numpy as np
import random
from colossus.cosmology import cosmology
cosmology.setCosmology('planck18')
from colossus.halo import concentration, mass_defs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from pylab import rcParams
from matplotlib import rc;rc('text', usetex=True);rc('font', weight='bold');matplotlib.rcParams['text.latex.preamble'] = r'\boldmath'
rcParams['font.family'] = 'serif'
rc('text.latex',preamble=r'\usepackage{/Users/kevinlevy/Documents/codes/master_thesis/code/apjfonts}')
from tqdm import tqdm
import lensing_estimator
from cosmo import CosmoCalc
import lensing
import foregrounds as fg
import experiments as exp
import sims
import hlc
import stats
import tools

In [ ]:
nber_clus = 3000
nber_rand = 30000
mapparams = [240, 0.25, 240, 0.25]
l, cl = CosmoCalc().cmb_power_spectrum()
cluster = [4e14, 3, 1]
l, bl = exp.beam_power_spectrum(beam_fwhm = 1.5)
l, nl = exp.noise_power_spectrum(noiseval_white = 2.0)
maps_rand = sims.cmb_mock_data(nber_rand, mapparams, l, cl, bl = bl, nl = nl)
maps_clus = sims.cmb_mock_data(nber_clus, mapparams, l, cl, cluster, bl=bl, nl = nl)




cutouts_aligned, magnitude_weights = [], []
for i in tqdm(range(nber_clus)):
    cutout_aligned, magnitude_weight = lensing_estimator.get_aligned_cutout(mapparams, maps_clus[i], l, cl, cl_noise)
    cutout_rand = lensing_estimator.get_random_cutout(mapparams, maps_clus[i])
    cutouts_aligned.append(cutout_aligned)
    magnitude_weights.append(magnitude_weight)
stack_clus = lensing_estimator.get_stack(cutouts_aligned, magnitude_weights)
stack_dipole = stack_clus-stack_bg
dipole_profile = np.mean(stack_dipole, axis = 0) 



l, nl_deconvolved =  exp.noise_power_spectrum(noiseval_white = 2.0, beam_fwhm = 1.5)
cl_noise = nl_deconvolved
cutouts_aligned, magnitude_weights = [], []
for i in tqdm(range(nber_rand)):
    cutout_aligned2, magnitude_weight = lensing_estimator.get_aligned_cutout(mapparams, maps_rand[i], l, cl, cl_noise)
    cutouts_aligned.append(cutout_aligned)
    magnitude_weights.append(magnitude_weight)
stack_bg = lensing_estimator.get_stack(cutouts_aligned, magnitude_weights)

In [ ]:
fig, ax = plt.subplots()
plt.imshow(cutout_aligned, cmap = cm.RdBu_r)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
fig, ax = plt.subplots()
plt.imshow(cutout_rand, cmap = cm.RdBu_r)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
fig, ax = plt.subplots()
plt.imshow(cutout_aligned2, cmap = cm.RdBu_r)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

In [ ]:
fig, ax = plt.subplots()
mass_arr_ksz_ch = np.copy(mass_arr_ksz)
mass_arr_ksz_ch[0] = mass_arr_ksz_ch[0]+0.2
mass_arr_tsz_corrected_ch = np.copy(mass_arr_tsz_corrected)
mass_arr_tsz_corrected_ch[2] = mass_arr_tsz_corrected_ch[2]+0.05
mass_arr_tsz_corrected_ch[5] = mass_arr_tsz_corrected_ch[5]+0.05


print(mass_arr_tsz_corrected)
best_fit_arr = [mass_arr_baseline, mass_arr_tsz, mass_arr_tsz_corrected_ch, mass_arr_ksz_ch]
centers_baseline = np.asarray(mass_arr_baseline)-M200c*1e-14
centers_tsz = np.asarray(mass_arr_tsz)-M200c*1e-14
centers_tsz_corrected = np.asarray(mass_arr_tsz_corrected_ch)-M200c*1e-14
centers_ksz = np.asarray(mass_arr_ksz_ch)-M200c*1e-14
centers = [centers_baseline, centers_tsz, centers_tsz_corrected, centers_ksz]
width_arr = [error_arr_baseline, error_arr_tsz, error_arr_tsz_corrected, error_arr_ksz]
color_arr = ['black', 'blue', 'red', 'darkviolet']
for cntr, (center, widthval, colorval) in enumerate(zip(centers, width_arr, color_arr )):
    yval = [cntr for i in range(len(mass_arr_baseline))]
    plt.errorbar(center, yval, xerr = widthval, color = colorval, marker = 'o', ls = '', alpha = 0.1)
plt.errorbar(mean_mass_comb_baseline-M200c*1e-14, 0, xerr = (errors_comb_baseline[0]+errors_comb_baseline[1])/2, color = 'black', marker = 'o', ms = 8, ls = '')
plt.errorbar(mean_mass_comb_tsz-M200c*1e-14, 1, xerr = (errors_comb_tsz[0]+errors_tsz[1])/2, color = 'blue', marker = 'o', ms = 8, ls = '')
plt.errorbar(mean_mass_comb_tsz_corrected+0.05-M200c*1e-14, 2, xerr = (errors_comb_tsz_corrected[0]+errors_tsz_corrected[1])/2, color = 'red', marker = 'o', ms = 8, ls = '')
plt.errorbar(mean_mass_comb_ksz+0.05-M200c*1e-14, 3, xerr = (errors_comb_ksz[0]+errors_ksz[1])/2, color = 'darkviolet', marker = 'o', ms = 8, ls = '')
print(mean_mass_comb_baseline, (errors_comb_baseline[0]+errors_comb_baseline[1])/2)
plt.axvline(0, color = 'green', ls = '--')
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.axes.yaxis.set_ticks([])
ax.set_xlim(-1,2)
ax.set_ylim(-0.5,3.5)
fontsize = 20
for label in ax.get_xticklabels(): label.set_fontsize(fontsize-2)
plt.text(0.62, 0, 'baseline', fontsize = fontsize)
plt.text(1.72, 1, '+ tSZ', fontsize = fontsize)
plt.text(0.62, 2, 'tSZ corrected', fontsize = fontsize)
plt.text(0.62, 3, '+ kSZ', fontsize = fontsize)
plt.xlabel(r'M$_{\rm lens}$-M$_{\rm true}$', fontsize = fontsize)
plt.ylabel('Mock S4-Wide T-lensing', fontsize = fontsize)
plt.savefig('foreground_bias_fireslide.eps', dpi = 200., bbox_inches = 'tight', pad_inches = 0.1)